# HATE SPEECH DETECTION FROM TWEETS

Sathvik Jamalpur, Aryan Karnati, Sai Madhav Peddoji

In [48]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
training_data=pd.read_csv('training_data.csv')
test_data=pd.read_csv('test_data.csv')

# Tf-Idf

In [47]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000)
l=[]
for i in training_data['Text']:
    l.append(''.join(j for j in i ))

train_tfidf = tfidf_vectorizer.fit_transform(l).toarray()
train_tfidf=pd.DataFrame(train_tfidf)

In [44]:
l=[]
for i in test_data['Text']:
    l.append(''.join(j+' ' for j in i ))
test_tfidf = tfidf_vectorizer.transform(l).toarray()
test_tfidf=pd.DataFrame(test_tfidf)

In [42]:
train_tfidf.to_csv('train_tfidf.csv')

In [45]:
test_tfidf.to_csv('test_tfidf.csv')

# Word2Vec

In [25]:
import gensim
model_w2v = gensim.models.Word2Vec(
            training_data['Text'],
            vector_size=200,
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model_w2v.train(training_data['Text'], total_examples= len(training_data['Text']), epochs=20)

(48301995, 241916100)

In [27]:
#for-training-data
l=[]
for i in training_data['Text']:
    l.append(''.join(j+' ' for j in i ))
df=pd.DataFrame(l,columns=['Text'])
l= df['Text'].apply(lambda x: x.split())

def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec
wordvec_arrays = np.zeros((len(l), 200)) 
for i in range(len(l)):
    wordvec_arrays[i,:] = word_vector(l[i], 200)
wordvec_train = pd.DataFrame(wordvec_arrays)


In [29]:
#for test-data
l=[]
for i in test_data['Text']:
    l.append(''.join(j+' ' for j in i ))


df=pd.DataFrame(l,columns=['Text'])
l= df['Text'].apply(lambda x: x.split())

wordvec_arrays = np.zeros((len(l), 200)) 
for i in range(len(l)):
    wordvec_arrays[i,:] = word_vector(l[i], 200)
wordvec_test = pd.DataFrame(wordvec_arrays)
